In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torchvision
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms

#For converting the dataset to torchvision dataset format
class VowelConsonantDataset(Dataset):
    def __init__(self, file_path,train=True,transform=None):
        self.transform = transform
        self.file_path=file_path
        self.train=train
        self.file_names=[file for _,_,files in os.walk(self.file_path) for file in files]
        self.len = len(self.file_names)
        if self.train:
            self.classes_mapping=self.get_classes()
    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self, index):
        file_name=self.file_names[index]
        image_data=self.pil_loader(self.file_path+"/"+file_name)
        if self.transform:
            image_data = self.transform(image_data)
        if self.train:
            file_name_splitted=file_name.split("_")
            Y1 = self.classes_mapping[file_name_splitted[0]]
            Y2 = self.classes_mapping[file_name_splitted[1]]
            z1,z2=torch.zeros(10),torch.zeros(10)
            z1[Y1-10],z2[Y2]=1,1
            label=torch.stack([z1,z2])

            return image_data, label

        else:
            return image_data, file_name
          
    def pil_loader(self,path):
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')

      
    def get_classes(self):
        classes=[]
        for name in self.file_names:
            name_splitted=name.split("_")
            classes.extend([name_splitted[0],name_splitted[1]])
        classes=list(set(classes))
        classes_mapping={}
        for i,cl in enumerate(sorted(classes)):
            classes_mapping[cl]=i
        return classes_mapping
    

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets

import torchvision.transforms as transforms

import numpy as np
import pandas as pd

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()])

In [ ]:
full_data = VowelConsonantDataset("../input/train/train",train=True,transform=transform)
train_size = int(0.9 * len(full_data))
test_size = len(full_data) - train_size

print(train_size)
print(test_size)

traindata, validationdata = random_split(full_data, [train_size, test_size])

trainloader = torch.utils.data.DataLoader(traindata, batch_size=1, shuffle=True)
validationloader = torch.utils.data.DataLoader(validationdata, batch_size=60, shuffle=True)

In [ ]:
testdata = VowelConsonantDataset("../input/test/test",train=False,transform=transform)
testloader = torch.utils.data.DataLoader(testdata, batch_size=60,shuffle=False)

In [ ]:
num_classes = 20

In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

print(images.shape)

print(images[0].shape)
print(labels[0])

In [ ]:
from torchvision import models

In [ ]:
vgg = models.vgg16_bn(pretrained = True).to(device)

In [ ]:
print(vgg)

In [ ]:
# change out features in in_built vgg model (1000 -> 10)
final_in_features = vgg.classifier[6].in_features
mod_classifier = list(vgg.classifier.children())[:-1]
mod_classifier.extend([nn.Linear(final_in_features, num_classes)])
print(mod_classifier)

In [ ]:
vgg.classifier = nn.Sequential(*mod_classifier)
print(vgg.classifier)

In [ ]:
vgg = vgg.to(device)

In [ ]:
batch_size = 1

In [102]:
def evaluation(dataloader, model):
    print(dataloader)
    #total, correct = 0, 0
    #for data in dataloader:
    inputs, labels = dataloader.next()
    print('inside')
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)
    print('Outputs : ',outputs)
    out1, pred1 = torch.max(outputs[0].data, 1)
    out2, pred2 = torch.max(outputs[1].data, 1)
    print('Out1 : ',out1)
    print('pred1 : ',pred1)
    print('Out2 : ',out2)
    print('pred2 : ',pred2)
    print(labels)
    print('----------------------------')
    print('--------------------------------')
        #total += labels.size(0)
        #correct += (pred == labels).sum().item()
        #print(pred==labels)
      #return 100 * correct / total

In [ ]:
dataiter = iter(trainloader)
evaluation(dataiter, vgg)

In [ ]:
class MyModel(nn.Module):
  def __init__(self):
    super(MyModel, self).__init__()
    self.cnn_model = nn.Sequential(
                     nn.Conv2d(3, 6, 5),               # (N, 3, 64, 64) -> (N, 6, 60, 60)
                     nn.Tanh(),
                     nn.AvgPool2d(2, stride=2),        # (N, 6, 60, 60) -> (N, 6, 30, 30)
                     nn.Conv2d(6, 16, 5),              # (N, 6, 30, 30) -> (N, 16, 26, 26)
                     nn.Tanh(),
                     nn.AvgPool2d(2, stride=2)         # (N, 16, 26, 26) -> (N, 16, 13, 13)
    )
    self.fc_model =  nn.Sequential(
                     nn.Linear(2704, 120),              # (N, 2704) -> (N, 120)
                     nn.Tanh(),
                     nn.Linear(120, 84),               # (N, 120) -> (N, 84)
                     nn.Tanh(),
                     nn.Linear(84, 10)                 # (N, 84) -> (N, 10)
    )
    
  def forward(self, x):
    #print(x.shape)
    x = self.cnn_model(x)
    print(x.shape)
    x = x.view(x.size(0), -1)
    print(x.shape)
    y = self.fc_model(x)
    x = self.fc_model(x)
    #print(x.shape)
    return x, y

In [ ]:
model = MyModel()

In [ ]:
model = model.to(device)

In [103]:
dataiter = iter(trainloader)
evaluation(dataiter, model)
#print(dataiter)

inside
torch.Size([1, 16, 13, 13])
torch.Size([1, 2704])
Outputs :  (tensor([[ 0.0256,  0.0493,  0.1078, -0.0389, -0.0663,  0.0416,  0.0684,  0.0676,
         -0.0163,  0.0948]], device='cuda:0', grad_fn=<AddmmBackward>), tensor([[ 0.0256,  0.0493,  0.1078, -0.0389, -0.0663,  0.0416,  0.0684,  0.0676,
         -0.0163,  0.0948]], device='cuda:0', grad_fn=<AddmmBackward>))
Out1 :  tensor([0.1078], device='cuda:0')
pred1 :  tensor([2], device='cuda:0')
Out2 :  tensor([0.1078], device='cuda:0')
pred2 :  tensor([2], device='cuda:0')
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]]], device='cuda:0')
----------------------------
--------------------------------
